# RAG – LlamaIndex + Simple Tool (EN/VN)
**Objective/Mục tiêu**: Build index from local docs and query; demonstrate a trivial tool (calculator).



# !pip install -q llama-index-core llama-index-embeddings-huggingface llama-index-vector-stores-faiss sentence-transformers transformers


In [ ]:

import os
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, ServiceContext, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss

os.makedirs("docs_li", exist_ok=True)
with open("docs_li/sample.txt", "w") as f:
    f.write("RAG augments LLMs by retrieving relevant text chunks and conditioning generation to improve faithfulness.")

reader = SimpleDirectoryReader("docs_li")
docs = reader.load_data()

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
vec_dim = embed_model._core.embedding_dim()
faiss_index = faiss.IndexFlatIP(vec_dim)
vector_store = FaissVectorStore(faiss_index=faiss_index)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(docs, storage_context=storage_context, embed_model=embed_model)

query_engine = index.as_query_engine(similarity_top_k=4)
resp = query_engine.query("Describe RAG in one line.")
print(resp)
